In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
#unzip Autopilot folder as data is inside there
!cp  /content/drive/Hackthon/McKinsey_Analytics/* .


In [4]:
!ls 

01_McKinsey_Analytics.ipynb
02_McKinsey_Analytics.ipynb
03_McKinsey_Analytics.ipynb
1000_double_half_adam_simple_Mckiney_model.h5
100_unbalance_adam_simple_Mckiney_model.h5
200_double_half_adam_simple_Mckiney_model.h5
200_unbalance_adam_simple_Mckiney_model.h5
AUC_simple_Mckiney_model.h5
CatBoost_LGBM_XGB_RF_Mckinsey_insurance_renewal_pridiction.ipynb
datalab
drive
feature_stand_100_lr_0001_07_drop_adam_cv_01780_model.h5
feature_stand_100_lr_0001_adam_simple_01784_Mckiney_model.h5
feature_stand_100_lr_001_07_drop_simple_Mckiney_model.h5
feature_stand_100_unbalance_adam_simple_Mckiney_model.h5
feature_stand_150_lr_0001_adam_simple_0_1778_Mckiney_model.h5
feature_stand_200_lr_0001_adamax_simple_Mckiney_model.h5
feature_stand_200_unbalance_adam_simple_Mckiney_model.h5
feature_stand_300_unbalance_adam_simple_Mckiney_model.h5
Mckinsey_final_submission_code_file.ipynb
sample_submission_sLex1ul.csv
simple_Mckiney_model.h5
test_66516Ee.csv
train_ZoGVYWq.csv


Install required Module like catboost lightgbm

In [5]:
!pip install catboost
!pip install ipywidgets
!pip install lightgbm
!pip install xgboost     

    100% |████████████████████████████████| 53.0MB 850kB/s 
    100% |████████████████████████████████| 112kB 4.3MB/s 
    100% |████████████████████████████████| 2.2MB 9.8MB/s 
    100% |████████████████████████████████| 737kB 9.2MB/s 



<h1>3. Exploratory Data Analysis</h1>
<h3>3.1. Import All required Classed first</h3>


In [6]:
import numpy as np
import pandas as pd
import os, shutil
from keras.models import Model
from keras import layers
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D,AveragePooling2D,BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense,Input,ELU
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from scipy import pi
import matplotlib.pyplot as plt
from keras.applications.inception_v3 import InceptionV3
from keras.layers import GlobalAveragePooling2D,GlobalMaxPooling2D
from keras.regularizers import l2
from keras.optimizers import Adam

from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras import layers
from keras.layers import Input, Dense, Dropout,LeakyReLU,BatchNormalization
from keras import backend as K
from sklearn.model_selection import train_test_split
import bisect
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix,roc_auc_score,mean_squared_error
from sklearn.metrics.classification import accuracy_score, log_loss
from sklearn.cross_validation import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from keras.models import load_model
from keras import regularizers
from keras.optimizers import Adam
from numpy import argmax
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier, Pool, cv
from lightgbm import LGBMClassifier

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [0]:
SEED = 0

In [0]:
Data = pd.read_csv("train_ZoGVYWq.csv")

In [9]:
print(Data.shape)
print(Data.columns)

(79853, 13)
Index(['id', 'perc_premium_paid_by_cash_credit', 'age_in_days', 'Income',
       'Count_3-6_months_late', 'Count_6-12_months_late',
       'Count_more_than_12_months_late', 'application_underwriting_score',
       'no_of_premiums_paid', 'sourcing_channel', 'residence_area_type',
       'premium', 'renewal'],
      dtype='object')


In [10]:
print(Data.describe())

                  id  perc_premium_paid_by_cash_credit   age_in_days  \
count   79853.000000                      79853.000000  79853.000000   
mean    57167.166368                          0.314288  18846.696906   
std     32928.970160                          0.334915   5208.719136   
min         2.000000                          0.000000   7670.000000   
25%     28640.000000                          0.034000  14974.000000   
50%     57262.000000                          0.167000  18625.000000   
75%     85632.000000                          0.538000  22636.000000   
max    114076.000000                          1.000000  37602.000000   

             Income  Count_3-6_months_late  Count_6-12_months_late  \
count  7.985300e+04           79756.000000            79756.000000   
mean   2.088472e+05               0.248671                0.078188   
std    4.965826e+05               0.691468                0.436507   
min    2.403000e+04               0.000000                0.000000   
2

Delete records which have empty in field "application_underwriting_score" (around 100 data is there)

In [11]:
values = {"Count_3-6_months_late":0,"Count_6-12_months_late":0,"Count_more_than_12_months_late":0,"application_underwriting_score":Data["application_underwriting_score"].mean()}
Data.fillna(value=values,inplace=True)
Data.shape

(79853, 13)

In [12]:
Data.isna().values.any()


False

In [0]:
#Combining months late feature
Data['count_month_late'] = 0.7*Data["Count_3-6_months_late"] + 0.25*Data["Count_6-12_months_late"]+0.05*Data["Count_more_than_12_months_late"]

Feature Standarization and Normalization

for Tree based model feature standarization is not required

Dealing with Categorical data

'age_in_days','sourcing_channel', 'residence_area_type',

In [0]:
  
Data["age_in_years_group"] = Data["age_in_days"]/(365)
Data["age_in_years_group"] =Data["age_in_years_group"].astype(int)


In [15]:
#check whether age_group convertion as expected
print(Data["age_in_years_group"].head(10))
print(Data["age_in_days"].head(10)/(365))

0    33
1    59
2    48
3    42
4    86
5    48
6    68
7    60
8    27
9    63
Name: age_in_years_group, dtype: int64
0    33.035616
1    59.030137
2    48.030137
3    42.030137
4    86.027397
5    48.019178
6    68.024658
7    60.030137
8    27.035616
9    63.035616
Name: age_in_days, dtype: float64


In [0]:
label_encoder_sc = LabelEncoder()
label_encoder_ar = LabelEncoder()
#sc_cat=to_categorical(label_encoder_sc.fit_transform(Data["sourcing_channel"]))
#ar_cat=to_categorical(label_encoder_ar.fit_transform(Data["residence_area_type"]))
#out_cat=to_categorical(label_encoder_out.fit_transform(Data["renewal"]))

Data["sourcing_channel_en"]=label_encoder_sc.fit_transform(Data["sourcing_channel"])
Data["residence_area_type_en"]=label_encoder_ar.fit_transform(Data["residence_area_type"])


In [0]:
data_xs = Data.drop(['renewal',"sourcing_channel",'residence_area_type','id','age_in_days'],axis=1)
data_xs_org = Data.drop(['renewal',"sourcing_channel_en",'residence_area_type_en','id','age_in_days'],axis=1)

In [18]:
data_xs.head()

,perc_premium_paid_by_cash_credit,Income,Count_3-6_months_late,Count_6-12_months_late,Count_more_than_12_months_late,application_underwriting_score,no_of_premiums_paid,premium,count_month_late,age_in_years_group,sourcing_channel_en,residence_area_type_en
0,0.429,355060,0.0,0.0,0.0,99.02,13,3300,0.0,33,2,1
1,0.010,315150,0.0,0.0,0.0,99.89,21,18000,0.0,59,0,1
2,0.917,84140,2.0,3.0,1.0,98.69,7,3300,2.2,48,2,0
3,0.049,250510,0.0,0.0,0.0,99.57,9,9600,0.0,42,0,1
4,0.052,198680,0.0,0.0,0.0,99.87,12,9600,0.0,86,1,1


In [0]:
balance_diff = int(len(Data["renewal"].loc[Data["renewal"]==1])/len(Data["renewal"].loc[Data["renewal"]==0]))

In [0]:
class_weight = {0:1,1:1} #0.8405
#class_weight = {0:8,1:0.5} # 1000 ep - auc 8916
#class_weight = {0:4,1:0.25} # 1000 ep - auc 8916

In [0]:

def model_evalution(model):
  print("####################### model Evalution started #######################")
  train_pre = model.predict(x_train)
  test_pre = model.predict(x_test)
  train_pro = model.predict_proba(x_train)
  test_pro = model.predict_proba(x_test)
  
  print("Train Accuracy: {0} \t Test Accuracy: {1}".format(accuracy_score(y_train, train_pre),accuracy_score(y_test,test_pre)))
  print("Train Loss: {0} \t Test Loss: {1}".format(mean_squared_error(y_train, train_pre),mean_squared_error(y_test,test_pre)))
  print("Train AUC: {0} \t Test AUC: {1}".format(roc_auc_score(y_train, train_pro[:,1]),roc_auc_score(y_test,test_pro[:,1])))
  print("Train Confusion Matrix: \n{0} \n Test Confusion Matrix: \n{1}".format(confusion_matrix(y_train, train_pre),confusion_matrix(y_test,test_pre)))
  

In [0]:
x_train,x_test,y_train,y_test = train_test_split(data_xs,Data["renewal"], test_size = 0.3,random_state=SEED)

In [28]:
rf_model = RandomForestClassifier()
rf_model.fit(x_train, y_train)
model_evalution(rf_model)

####################### model Evalution started #######################
Train Accuracy: 0.9963862103511817 	 Test Accuracy: 0.9328769410586074
Train Loss: 0.0036137896488183624 	 Test Loss: 0.06712305894139255
Train AUC: 0.9997568172628787 	 Test AUC: 0.7551355704815889
Train Confusion Matrix: 
[[ 3354   165]
 [   37 52341]] 
 Test Confusion Matrix: 
[[  295  1184]
 [  424 22053]]


In [0]:
# here we can see difference in test and train accuracy is high:- model is overfitted

In [97]:
xgb_model = XGBClassifier()
xgb_model.fit(x_train, y_train)
model_evalution(xgb_model)

####################### model Evalution started #######################


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Train Accuracy: 0.9413743134694169 	 Test Accuracy: 0.9400150275505093
Train Loss: 0.05862568653058304 	 Test Loss: 0.059984972449490734
Train AUC: 0.8538967848543557 	 Test AUC: 0.841084987995993
Train Confusion Matrix: 
[[  490  3029]
 [  248 52130]] 
 Test Confusion Matrix: 
[[  181  1298]
 [  139 22338]]


In [0]:
# here we can see difference in test and train accuracy is very less but in confusion matrix more wrong predicted :- model is Underfitted

In [101]:
xgb_hist_model = XGBClassifier(tree_method ='hist')
xgb_hist_model.fit(x_train, y_train)
model_evalution(xgb_hist_model)

####################### model Evalution started #######################


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Train Accuracy: 0.9411059627529206 	 Test Accuracy: 0.9404742027049591
Train Loss: 0.05889403724707945 	 Test Loss: 0.05952579729504091
Train AUC: 0.8539870146939709 	 Test AUC: 0.8412703325942109
Train Confusion Matrix: 
[[  491  3028]
 [  264 52114]] 
 Test Confusion Matrix: 
[[  192  1287]
 [  139 22338]]


In [102]:
lgbm_model = LGBMClassifier()
lgbm_model.fit(x_train, y_train)
model_evalution(lgbm_model)

####################### model Evalution started #######################


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Train Accuracy: 0.9466697676082795 	 Test Accuracy: 0.9385957588913008
Train Loss: 0.05333023239172049 	 Test Loss: 0.061404241108699284
Train AUC: 0.9079599428774747 	 Test AUC: 0.8356898704025688
Train Confusion Matrix: 
[[  697  2822]
 [  159 52219]] 
 Test Confusion Matrix: 
[[  182  1297]
 [  174 22303]]


In [103]:
catboost_model = CatBoostClassifier(verbose=False)
catboost_model.fit(x_train, y_train)
model_evalution(catboost_model)

Learning rate set to 0.052316
####################### model Evalution started #######################
Train Accuracy: 0.9549528597241355 	 Test Accuracy: 0.938762731674737
Train Loss: 0.04504714027586454 	 Test Loss: 0.06123726832526298
Train AUC: 0.8840175110885153 	 Test AUC: 0.8381529396303029
Train Confusion Matrix: 
[[ 1038  2481]
 [   37 52341]] 
 Test Confusion Matrix: 
[[  208  1271]
 [  196 22281]]


In [104]:
voting_model = VotingClassifier(estimators=[('rf', rf_model),
                                       ('xgb', xgb_model), 
                                       ('xgb_hist_model', xgb_hist_model),
                                       ('catboost', catboost_model),
                                       ('lgbm', lgbm_model)], 
                           voting='soft')
voting_model.fit(x_train, y_train)
model_evalution(voting_model)

Learning rate set to 0.052316
####################### model Evalution started #######################


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Train Accuracy: 0.9519473316993756 	 Test Accuracy: 0.939305393220905
Train Loss: 0.048052668300624365 	 Test Loss: 0.06069460677909501
Train AUC: 0.9770300089005869 	 Test AUC: 0.8353688751566736
Train Confusion Matrix: 
[[  883  2636]
 [   50 52328]] 
 Test Confusion Matrix: 
[[  176  1303]
 [  151 22326]]


In [0]:

data_xs_org = Data.drop(['perc_premium_paid_by_cash_credit','renewal',"sourcing_channel_en",'residence_area_type_en','id','age_in_days'],axis=1)
x_train,x_test,y_train,y_test = train_test_split(data_xs_org,Data["renewal"], test_size = 0.3,random_state=SEED)
x_train.head()
categorical_features_indices = np.array([6,7])

In [82]:
categorical_features_indices = np.array([6,7])
params = {'iterations':[50],
          'eval_metric':["AUC"],
          'learning_rate':[1,0.1,0.01,0.001],
          'depth':[2,4,6,8,10],
          'verbose':[True],
          'class_weights':[[14,1],[1,1]],
          #'l2_leaf_reg':[10,1,0.1,0.01,0.001,0.0001],
          #'bagging_temperature':[0,0.1,0.5,0,0.01,0.07],
         }

rf_model = CatBoostClassifier()
grid = GridSearchCV(rf_model,params,scoring="roc_auc")
grid.fit(x_train, y_train,
         cat_features=categorical_features_indices,
         eval_set=(x_test, y_test),)
print(grid.best_params_)
model_evalution(grid)

0:	test: 0.7383569	best: 0.7383569 (0)	total: 91.7ms	remaining: 4.49s
1:	test: 0.7733960	best: 0.7733960 (1)	total: 170ms	remaining: 4.07s
2:	test: 0.7930623	best: 0.7930623 (2)	total: 260ms	remaining: 4.08s
3:	test: 0.8013663	best: 0.8013663 (3)	total: 345ms	remaining: 3.96s
4:	test: 0.8057674	best: 0.8057674 (4)	total: 441ms	remaining: 3.97s
5:	test: 0.8071384	best: 0.8071384 (5)	total: 536ms	remaining: 3.93s
6:	test: 0.8071384	best: 0.8071384 (5)	total: 617ms	remaining: 3.79s
7:	test: 0.8074229	best: 0.8074229 (7)	total: 708ms	remaining: 3.72s
8:	test: 0.8085843	best: 0.8085843 (8)	total: 798ms	remaining: 3.63s
9:	test: 0.8085033	best: 0.8085843 (8)	total: 891ms	remaining: 3.56s
10:	test: 0.8087151	best: 0.8087151 (10)	total: 984ms	remaining: 3.49s
11:	test: 0.8105559	best: 0.8105559 (11)	total: 1.07s	remaining: 3.4s
12:	test: 0.8110474	best: 0.8110474 (12)	total: 1.17s	remaining: 3.32s
13:	test: 0.8110474	best: 0.8110474 (12)	total: 1.26s	remaining: 3.23s
14:	test: 0.8115244	best: 

In [77]:
model = CatBoostClassifier(
    random_seed=20,
    logging_level='Silent',
    iterations=200,
    eval_metric="AUC",
    learning_rate=0.5,
    depth=4
)
model.fit(
    x_train, y_train,
    cat_features=categorical_features_indices,
    eval_set=(x_test, y_test),
    #logging_level='Verbose',  # you can uncomment this for text output
    plot=True,
    
);
print(model.feature_importances_)
print(model.get_param)

model_evalution(model)

MetricVisualizer(data={'catboost_info': {'path': 'catboost_info', 'name': 'catboost_info', 'content': {'passed…

[3.64318791e+00 1.98682477e-01 1.41580969e+01 7.27877779e+00
 1.18832509e+01 7.17452685e+00 7.24948887e-01 3.02464741e-02
 6.54010919e-01 3.99845370e+01 1.42697338e+01]
<bound method CatBoost.get_param of <catboost.core.CatBoostClassifier object at 0x7fa55214dba8>>
####################### model Evalution started #######################
Train Accuracy: 0.9409270622752562 	 Test Accuracy: 0.9397228251794958
Train Loss: 0.05907293772474372 	 Test Loss: 0.06027717482050426
Train AUC: 0.8212157984500954 	 Test AUC: 0.8172086089775852
Train Confusion Matrix: 
[[  493  3026]
 [  276 52102]] 
 Test Confusion Matrix: 
[[  175  1304]
 [  140 22337]]


In [0]:
params = {'n_estimators':[50,100,200,300,400],
         'max_depth':[2,3,4,5,6,7,8],
          'class_weight':[{0: 10, 1: 1},{0: 14, 1: 1},{0: 7, 1: 1},{0:4, 1: 1}]          
         }

In [48]:
rf_model = RandomForestClassifier()
grid = GridSearchCV(rf_model,params,scoring="roc_auc")
grid.fit(x_train, y_train)
print(grid.best_params_)
model_evalution(grid)

{'class_weight': {0: 4, 1: 1}, 'max_depth': 8, 'n_estimators': 200}
####################### model Evalution started #######################
Train Accuracy: 0.9296384421346405 	 Test Accuracy: 0.9193521456002671
Train Loss: 0.0703615578653595 	 Test Loss: 0.08064785439973285
Train AUC: 0.8690433969232617 	 Test AUC: 0.8392831461131796
Train Confusion Matrix: 
[[ 1634  1885]
 [ 2048 50330]] 
 Test Confusion Matrix: 
[[  601   878]
 [ 1054 21423]]


In [0]:
params = {'n_estimators':[50,100,200,300,400],
         'max_depth':[2,3,4,5,6,7,8],
          "min_samples_leaf":[0.1,0.3,0.5,1],
          "min_weight_fraction_leaf":[0,0.3,0.5],
          
          #'class_weight':["balanced",{0:1,1:1}]          
         }

In [0]:
dict_keys(['bootstrap', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes',
           'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])
---------------------------------------------------------------------------
ValueError     

In [64]:
rf_model = RandomForestClassifier()
#print(rf_model.get_params().keys())
grid = GridSearchCV(rf_model,params,scoring="accuracy")
grid.fit(x_train, y_train)
print(grid.best_params_)
model_evalution(grid)

{'max_depth': 8, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0, 'n_estimators': 200}
####################### model Evalution started #######################
Train Accuracy: 0.945310123978031 	 Test Accuracy: 0.9403072299215228
Train Loss: 0.05468987602196898 	 Test Loss: 0.05969277007847721
Train AUC: 0.863101378680048 	 Test AUC: 0.8381164963972036
Train Confusion Matrix: 
[[  551  2968]
 [   89 52289]] 
 Test Confusion Matrix: 
[[  142  1337]
 [   93 22384]]


In [47]:
y_pre = xgb_model.predict(x_test)
y_pre_train = xgb_model.predict(x_train)


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [48]:
y_pre[1]

1

In [50]:
print(accuracy_score(y_train, y_pre_train))
print(accuracy_score(y_test, y_pre))

0.9413743134694169
0.9400150275505093


In [46]:
confusion_matrix(y_test,y_pre)

array([[  181,  1298],
       [  139, 22338]])

In [0]:
param_test4 = {
    'subsample':[i/10.0 for i in range(6,10)],
    'colsample_bytree':[i/10.0 for i in range(6,10)]
}


In [35]:
x_model = XGBClassifier(
    learning_rate =0.1,
    n_estimators=1000,
    max_depth = 3,
    min_child_weight = 5,
    gamma=0.4,
    subsample=0.55,
    colsample_bytree=0.85,
    reg_alpha=0.005,
    objective= 'binary:logistic',
    nthread=4,
    scale_pos_weight=1,
    seed=SEED,
    ver
)
x_model.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.85, gamma=0.4, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=5, missing=None,
       n_estimators=1000, n_jobs=1, nthread=4, objective='binary:logistic',
       random_state=0, reg_alpha=0.005, reg_lambda=1, scale_pos_weight=1,
       seed=27, silent=True, subsample=0.55)

In [0]:

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(x_train, y_train)
print(rf_random.best_params_)

In [0]:
rf_random.best_params_

In [67]:
rf = RandomForestClassifier(n_estimators= 1000,
               max_features= 'sqrt',
               max_depth= 10,
               min_samples_split= 10,
               min_samples_leaf= 4,
               bootstrap= True)
rf.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=4, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [0]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [0]:
base_model = RandomForestRegressor(n_estimators = 10, random_state = 42)
base_model.fit(train_features, train_labels)
base_accuracy = evaluate(base_model, x_test, y_test)

best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, test_features, test_labels)

print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

In [0]:
y_pre = rf.predict(x_test)

In [0]:
y_pre_cat = list(map(lambda x:1 if x>0.5 else 0,y_pre))

In [70]:
accuracy_score(y_test, y_pre_cat)

0.9398063115712139

In [52]:
confusion_matrix(y_test,y_pre_cat)

array([[  189,  1311],
       [  150, 22306]])

In [53]:
print(model.evaluate(data_xs,Data["renewal"]))
y_pre = model.predict(x_test)

79853/79853 [==============================] - 5s 59us/step
[0.17816876518791736, 0.9393385345579953, 0.8354461366166818]


In [0]:
Data["Renewal_pre"] = y_pre

In [54]:
print(y_pre.max())
print(y_pre.min())
print(y_pre.mean())

0.99710613
0.12731105
0.9327458


In [55]:
np.percentile(y_pre,[1,10,14,25,50,75,99])

array([0.54599284, 0.83395025, 0.87505872, 0.92148498, 0.96912733,
       0.98485631, 0.99466812])

In [0]:
y_pre_0=[]
y_pre_1=[]
for i,j in zip(y_test,y_pre):
  if i == 0:
    y_pre_0.append(j)
  else:
    y_pre_1.append(j)
    
y_pre_0 =np.asanyarray(y_pre_0)
y_pre_1 =np.asanyarray(y_pre_1)

In [60]:
print(np.percentile(y_pre_0,[10]))
print(np.percentile(y_pre_1,[3]))

[0.55052943]
[0.72605385]


In [61]:
print(y_pre_1.max())
print(y_pre_1.min())
print(y_pre_1.mean())


print(y_pre_0.max())
print(y_pre_0.min())
print(y_pre_0.mean())

0.99710613
0.12731105
0.9426463
0.99549365
0.19546139
0.7845285


In [0]:
y_pre_cat = list(map(lambda x:1 if x>0.5 else 0,y_pre))

In [63]:
y_pre_cat[:10]

[1, 1, 1, 1, 1, 0, 1, 1, 1, 1]

In [65]:
confusion_matrix(y_test,y_pre_cat)

array([[   78,  1422],
       [   35, 22421]])

In [0]:
import math
def calculate_per_improve(incentive):
  extra_hr = 10*(1- math.exp(-(incentive/400)))
  per_imp = 20*(1- math.exp(-(extra_hr/5)))
  return round(per_imp/100,4)
 
calculate_per_improve(1000)

def calculate_revenue_gain(p_benchmark,per_improve,premium_on_policy,incentive):
  return ((p_benchmark + p_benchmark*per_improve)*premium_on_policy-incentive)




In [0]:
incentive_list =  [i for i in range(50,1600,50)]
per_improve_list = list(map(calculate_per_improve,incentive_list))
print(per_improve_list)
def predict_incentive(p_benchmark,premium_on_policy):
    revenue_gain_no_incentive=calculate_revenue_gain(p_benchmark,0,premium_on_policy,0)
    best_incentive = 0
    for i,p in zip(incentive_list,per_improve_list):
        revenue_gain = calculate_revenue_gain(p_benchmark,p,premium_on_policy,i)
        if revenue_gain_no_incentive < revenue_gain:
            if (p_benchmark + p_benchmark*p)<1:
              revenue_gain_no_incentive =revenue_gain
              best_incentive = i
    return best_incentive

[0.0419, 0.0715, 0.093, 0.109, 0.121, 0.1304, 0.1377, 0.1435, 0.1482, 0.152, 0.1551, 0.1577, 0.1599, 0.1617, 0.1632, 0.1645, 0.1656, 0.1666, 0.1674, 0.1681, 0.1687, 0.1692, 0.1697, 0.1701, 0.1704, 0.1708, 0.171, 0.1712, 0.1715, 0.1716, 0.1718]


In [0]:
predict_incentive(0.95,100000)

50